In [1]:
import sys

sys.path.append("../../")
import os
import pprint
import logging
import argparse

from shutil import copyfile

import yaml
import numpy as np
import torch
import torch.optim as optim

from accuracy import Accuracy, Accuracy3D
from models.pretrained_net import *

from common.batcher.batch_processor import BatchProcessor4D
from common.datasets_parsers.av_parser import AVDBParser
from common.group_random_sampler import GroupRandomSampler
from common.image_processor import TorchImageProcessor
from common.images_batcher import ImagesBatcher
from common.logging.logging_control import create_logger
from common.lr_schedulers import *
from common.losses import *
from common.net_trainer import NetTrainer


def get_net_params(net, lr, weight_decay):
    # large lr for last fc parameters
    params = []
    for name, value in net.named_parameters():
        if not value.requires_grad:
            continue
        if "bias" in name:
            params += [{"name": name, "params": value, "lr": lr, "weight_decay": 0}]
        else:
            params += [
                {"name": name, "params": value, "lr": lr, "weight_decay": weight_decay}
            ]
    return params


def train():
    '''
    arguments_parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    arguments_parser.add_argument(
        "--config", help="yml config path", type=str, required=True
    )
    arguments_parser.add_argument(
        "--host", help="name of the host server", type=str, required=True
    )
    '''

    #args = arguments_parser.parse_args()

    # get host name for brevity
    #host = args.host
    host = "local"
    config_path = "A:/Профиль/Rab Table/Учёба/3/нейронные сети/4/pytorch/VEmotionNet/config.yml"

    # read configuration
    with open(config_path) as yml_file:
        cfg = yaml.load(Loader=yaml.FullLoader, stream=yml_file)
        params = {
            key: cfg[key]
            for key in cfg
            if not key in ["dataset", "ini_net", "logging", "test"]
        }
        dataset = {key: cfg["dataset"][key] for key in cfg["dataset"]}
        log = {key: cfg["logging"][key] for key in cfg["logging"]}
        ini_net = cfg["ini_net"]

    # init logging
    experiment_name = params["train"]["experiment_name"] + str(
        params["train"]["cuda_device"]
    )
    create_logger(
        log["log_dir"],
        experiment_name + ".log",
        console_level=logging.CRITICAL,
        file_level=logging.NOTSET,
    )

    # log configuration
    pp = pprint.PrettyPrinter(indent=4)
    logging.info("Configuration is:\n%s" % pp.pformat(cfg))
    experiment_path = os.path.join(log["log_dir"], params["train"]["experiment_name"])
    if not os.path.exists(experiment_path):
        os.makedirs(experiment_path)
    copyfile(config_path, os.path.join(experiment_path, "config.yml"))

    device = torch.device(
        "cuda:" + str(params["train"]["cuda_device"])
        if torch.cuda.is_available()
        else "cpu"
    )
    torch.manual_seed(params["seed"])
    torch.cuda.manual_seed_all(params["seed"])

    train_dataset_parser = AVDBParser(
        dataset_root=dataset["train"]["data_root"],
        file_list=dataset["train"]["file_list"],
        max_num_clips=params["parser"]["max_num_clips"],
        max_num_samples=params["parser"]["max_num_samples"],
        ungroup=params["preproc"]["data_frame"]["depth"] == 1,
        normalize="AFEW-VA" in dataset["train"]["data_root"],
    )
    valid_dataset_parser = AVDBParser(
        dataset_root=dataset["valid"]["data_root"],
        file_list=dataset["valid"]["file_list"],
        max_num_clips=params["parser"]["max_num_clips"],
        max_num_samples=params["parser"]["max_num_samples"],
        ungroup=params["preproc"]["data_frame"]["depth"] == 1,
        normalize="AFEW-VA" in dataset["train"]["data_root"],
    )

    softmax_size = (
        params["net"]["softmax_size"]
        if params["net"]["softmax_size"] > 0
        else train_dataset_parser.get_class_num()
    )
    print("\rsoftmax_size = %d" % softmax_size)

    # create train data sampler
    train_data_sampler = GroupRandomSampler(
        data=train_dataset_parser.get_data(),
        num_sample_per_classes=params["preproc"]["data_frame"]["depth"],
        samples_is_randomize=params["sampler"]["samples_is_randomize"],
        step_size_for_samples=params["sampler"]["step_size_for_samples"],
    )
    valid_data_sampler = GroupRandomSampler(
        data=valid_dataset_parser.get_data(),
        num_sample_per_classes=params["preproc"]["data_frame"]["depth"],
        samples_is_randomize=params["sampler"]["samples_is_randomize"],
        step_size_for_samples=params["sampler"]["step_size_for_samples"],
        is_shuffle=False,
    )

    # create train image processor
    train_image_processor = TorchImageProcessor(
        image_size=[
            params["preproc"]["data_frame"]["width"],
            params["preproc"]["data_frame"]["height"],
        ],
        is_color=params["preproc"]["is_color"],
        mean=params["preproc"]["mean"],
        scale=params["preproc"]["scale"],
        crop_size=params["preproc"]["crop_size"],
        pad=params["preproc"]["aug"]["pad"],
        use_cutout=params["preproc"]["aug"]["use_cutout"],
        use_mirroring=params["preproc"]["aug"]["use_mirroring"],
        use_random_crop=params["preproc"]["aug"]["use_random_crop"],
        use_center_crop=params["preproc"]["aug"]["use_center_crop"],
    )
    valid_image_processor = TorchImageProcessor(
        image_size=[
            params["preproc"]["data_frame"]["width"],
            params["preproc"]["data_frame"]["height"],
        ],
        is_color=params["preproc"]["is_color"],
        mean=params["preproc"]["mean"],
        scale=params["preproc"]["scale"],
        crop_size=params["preproc"]["crop_size"],
        pad=params["preproc"]["aug"]["pad"],
        use_cutout=False,
        use_mirroring=False,
        use_random_crop=False,
        use_center_crop=params["preproc"]["aug"]["pad"] > 0
        or params["preproc"]["crop_size"] != params["preproc"]["data_frame"]["width"]
        or params["preproc"]["crop_size"] != params["preproc"]["data_frame"]["height"],
    )

    # create train image batcher
    train_images_batcher = ImagesBatcher(
        queue_size=params["train_batcher"]["queue_size"],
        batch_size=params["train_batcher"]["batch"],
        data_sampler=train_data_sampler,
        image_processor=train_image_processor,
        single_epoch=False,
        cache_data=False,
        disk_reader_process_num=params["train_batcher"]["disk_reader_process_num"],
    )
    valid_images_batcher = ImagesBatcher(
        queue_size=params["valid_batcher"]["queue_size"],
        batch_size=params["valid_batcher"]["batch"],
        data_sampler=valid_data_sampler,
        image_processor=valid_image_processor,
        single_epoch=True,
        cache_data=False,
        disk_reader_process_num=params["valid_batcher"]["disk_reader_process_num"],
    )

    # create batch processor
    batch_processor = BatchProcessor4D(
        depth=params["preproc"]["data_frame"]["depth"],
        use_pin_memory=params["batch_proc"]["use_pin_memory"],
        cuda_id=params["train"]["cuda_device"],
        use_async=params["batch_proc"]["use_async"],
    )

    net_type = params["net"]["type"]
    if net_type == "ResNet":
        net = CNNNet(
            softmax_size,
            depth=params["net"]["depth"],
            data_size=params["preproc"]["data_frame"],
            pretrain_weight="resnet-34-kinetics.pth",
        )
    else:
        print("Type net is not supported!")
        exit(0)

    loss = TotalLoss(params["losses"], 1, params["train"]["cuda_device"])

    net.to(device)
    loss.to(device)

    if params["net"]["fine_tune"]:
        net.load_state_dict(torch.load(ini_net))

    lr = params["opt"]["lr"]
    momentum = params["opt"]["momentum"]
    weight_decay = params["opt"]["weight_decay"]

    opt_type = params["opt"]["type"]
    if opt_type == "SGD":
        optimizer = optim.SGD(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            momentum=momentum,
            weight_decay=weight_decay,
            nesterov=True,
        )  # , caffe_like=True)
    if opt_type == "ASGD":
        optimizer = optim.ASGD(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )  # , caffe_like=True)
    if opt_type == "Adagrad":
        optimizer = optim.Adagrad(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )
    if opt_type == "Adadelta":
        optimizer = optim.Adadelta(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )
    if opt_type == "Adam":
        optimizer = optim.Adam(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )
    if opt_type == "RMSprop":
        optimizer = optim.RMSprop(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
            momentum=momentum,
        )
    if opt_type == "LBFGS":
        optimizer = optim.LBFGS(
            net.parameters(),
            lr=lr,
            max_iter=5,
            max_eval=None,
            tolerance_grad=1e-05,
            tolerance_change=1e-09,
            history_size=1,
            line_search_fn=None,
        )

    # if params['net']['init'] == 'fine_tune':
    #    optimizer = cp['optimizer']

    lr_scheduler_type = params["lr_scheduler"]["type"]
    gamma = float(params["lr_scheduler"]["gamma"])
    if lr_scheduler_type == "ReduceLROnPlateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=gamma)
    if lr_scheduler_type == "MultiStepLR":
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[15000, 30000, 60000], gamma=gamma
        )
    if lr_scheduler_type == "ExponentialLR":
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    if lr_scheduler_type == "SGDR":
        scheduler = SGDR_scheduler(
            optimizer,
            lr_start=lr,
            lr_end=gamma * lr,
            lr_period=params["train"]["epoch_size"] // params["train"]["step_size"],
            scale_lr=params["lr_scheduler"]["scale_lr"],
            scale_lr_fc=params["lr_scheduler"]["scale_lr_fc"],
        )

    net_trainer = NetTrainer(
        logs_dir=log["tb_log_dir"],
        cuda_id=params["train"]["cuda_device"],
        experiment_name=experiment_name,
        snapshot_dir=log["snapshot_dir"],
        config_data=cfg,
    )

    if params["preproc"]["data_frame"]["depth"] == 1:
        accuracy_fn = Accuracy(valid_dataset_parser.get_data())
    else:
        accuracy_fn = Accuracy3D(
            valid_dataset_parser.get_data(),
            depth=params["preproc"]["data_frame"]["depth"],
        )

    # lets try to train
    torch.set_num_threads(2)
    net_trainer.train(
        model=net,
        optimizer=optimizer,
        scheduler=scheduler,
        train_data_batcher=train_images_batcher,
        val_data_batcher=valid_images_batcher,
        val_iter=params["train"]["validate_iter"],
        batch_processor=batch_processor,
        loss_function=loss,
        max_iter=params["train"]["max_iter"],
        step_size=params["train"]["step_size"],
        snapshot_iter=params["train"]["snapshot_iter"],
        step_print=params["train"]["step_print"],
        accuracy_function=accuracy_fn,
    )

    # finalize data loader queues
    train_images_batcher.finish()
    valid_images_batcher.finish()


def test():
    '''
    arguments_parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    arguments_parser.add_argument(
        "--config", help="yml config path", type=str, required=True
    )
    arguments_parser.add_argument(
        "--host", help="name of the host server", type=str, required=True
    )
    '''

    #args = arguments_parser.parse_args()

    # get host name for brevity
    #host = args.host
    host = "local"
    config_path = "A:/Профиль/Rab Table/Учёба/3/нейронные сети/4/pytorch/VEmotionNet/config.yml"

    # read configuration
    with open(config_path) as yml_file:
        cfg = yaml.load(Loader=yaml.FullLoader, stream=yml_file)
        train_params = {
            key: cfg[key]
            for key in cfg
            if not key in ["dataset", "ini_net", "logging", "test"]
        }
        test_params = cfg["test"]
        dataset = {
            key: test_params["dataset"][key] for key in test_params["dataset"]
        }

    device = torch.device(
        "cuda:" + str(test_params["cuda_device"]) if torch.cuda.is_available() else "cpu"
    )
    torch.manual_seed(train_params["seed"])
    torch.cuda.manual_seed_all(train_params["seed"])
    cp = torch.load(test_params["file_model"])

    test_dataset_parser = AVDBParser(
        dataset_root=dataset["data_root"],
        file_list=dataset["test_file_list"],
        ungroup=train_params["preproc"]["data_frame"]["depth"] == 1,
    )

    test_data_sampler = GroupRandomSampler(
        data=test_dataset_parser.get_data(),
        num_sample_per_classes=train_params["preproc"]["data_frame"]["depth"],
        samples_is_randomize=train_params["sampler"]["samples_is_randomize"],
        step_size_for_samples=train_params["sampler"]["step_size_for_samples"],
        is_shuffle=False,
    )

    test_image_processor = TorchImageProcessor(
        image_size=[
            train_params["preproc"]["data_frame"]["width"],
            train_params["preproc"]["data_frame"]["height"],
        ],
        is_color=train_params["preproc"]["is_color"],
        mean=train_params["preproc"]["mean"],
        scale=train_params["preproc"]["scale"],
        crop_size=train_params["preproc"]["crop_size"],
        pad=train_params["preproc"]["aug"]["pad"],
        use_cutout=False,
        use_mirroring=False,
        use_random_crop=False,
        use_center_crop=train_params["preproc"]["aug"]["pad"] > 0
        or train_params["preproc"]["crop_size"]
        != train_params["preproc"]["data_frame"]["width"]
        or train_params["preproc"]["crop_size"]
        != train_params["preproc"]["data_frame"]["height"],
    )

    test_images_batcher = ImagesBatcher(
        queue_size=train_params["valid_batcher"]["queue_size"],
        batch_size=train_params["valid_batcher"]["batch"],
        data_sampler=test_data_sampler,
        image_processor=test_image_processor,
        single_epoch=True,
        cache_data=False,
        disk_reader_process_num=train_params["valid_batcher"]["disk_reader_process_num"],
    )

    batch_processor = BatchProcessor4D(
        depth=train_params["preproc"]["data_frame"]["depth"],
        use_pin_memory=train_params["batch_proc"]["use_pin_memory"],
        cuda_id=test_params["cuda_device"],
        use_async=False,
    )

    model = cp["model"]
    model.to(device)
    model.eval()

    test_images_batcher.start()
    predict = np.zeros((0, 2), dtype=np.float32)
    while True:
        batch = test_images_batcher.next_batch()
        if batch is None:
            break

        data, _, _ = batch_processor.pre_processing(batch)
        logits = model(data)
        predict = np.concatenate((predict, logits.cpu().data.numpy()), axis=0)

    accuracy_fn = Accuracy(test_dataset_parser.get_data())
    accuracy_fn.by_frames(predict)
    accuracy_fn.by_clips(predict)


if __name__ == "__main__":
    train()
    #test()


AVDB meta parsing:  89%|████████▉ | 38260/42956 [00:04<00:00, 9118.10images/s] 


softmax_size = 2
disk_reader_process_num: 1
disk_reader_process_num: 1


2023-12-14 10:44:39,604:CRITICAL:EmoV2_step40: iteration: 0: Loss: 0.202574, lr: 0.009000
c:\Users\artem\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
2023-12-14 10:46:37,924:CRITICAL:EmoV2_step40: validate. Iteration: 0: Accuracy (valence, arousal): 2.970% 21.782%
2023-12-14 10:46:37,926:CRITICAL:EmoV2_step40: validate. Iteration: 0: Loss: 0.251422


targets = (38261, 2), predict = (38261, 2)
target = torch.Size([38261]), pred = torch.Size([38261])
Valence:
   accuracy per frames: 2.339%
   error per frames: frames=0.437, std=0.216
   concordance correlation coefficient per frames: nan
target = torch.Size([38261]), pred = torch.Size([38261])
Arousal:
   accuracy per frames: 20.935%
   error per frames: frames=0.318, std=0.216
   concordance correlation coefficient per frames: nan
---------

targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Valence:
   accuracy per clips: 2.970%
   error per clips: mean=0.441, std=0.212
targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Arousal:
   accuracy per clips: 21.782%
   error per clips: mean=0.302, std=0.220
---------



2023-12-14 10:46:41,479:CRITICAL:EmoV2_step40: iteration: 10: Loss: 0.258353, lr: 0.009000
2023-12-14 10:46:45,547:CRITICAL:EmoV2_step40: iteration: 20: Loss: 0.244511, lr: 0.009000
2023-12-14 10:46:49,776:CRITICAL:EmoV2_step40: iteration: 30: Loss: 0.265437, lr: 0.009000
2023-12-14 10:46:53,768:CRITICAL:EmoV2_step40: iteration: 40: Loss: 0.240334, lr: 0.009000
2023-12-14 10:46:57,792:CRITICAL:EmoV2_step40: iteration: 50: Loss: 0.263831, lr: 0.009000
2023-12-14 10:47:01,711:CRITICAL:EmoV2_step40: iteration: 60: Loss: 0.281112, lr: 0.009000
2023-12-14 10:47:06,054:CRITICAL:EmoV2_step40: iteration: 70: Loss: 0.250379, lr: 0.009000
2023-12-14 10:47:09,888:CRITICAL:EmoV2_step40: iteration: 80: Loss: 0.262570, lr: 0.009000
2023-12-14 10:47:13,824:CRITICAL:EmoV2_step40: iteration: 90: Loss: 0.273164, lr: 0.009000
2023-12-14 10:47:18,248:CRITICAL:EmoV2_step40: iteration: 100: Loss: 0.267271, lr: 0.009000
2023-12-14 10:49:09,724:CRITICAL:EmoV2_step40: validate. Iteration: 100: Accuracy (valenc

...

targets = (38261, 2), predict = (38261, 2)
target = torch.Size([38261]), pred = torch.Size([38261])
Valence:
   accuracy per frames: 2.339%
   error per frames: frames=0.437, std=0.216
   concordance correlation coefficient per frames: nan
target = torch.Size([38261]), pred = torch.Size([38261])
Arousal:
   accuracy per frames: 20.935%
   error per frames: frames=0.318, std=0.216
   concordance correlation coefficient per frames: nan
---------

targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Valence:
   accuracy per clips: 2.970%
   error per clips: mean=0.441, std=0.212
targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Arousal:
   accuracy per clips: 21.782%
   error per clips: mean=0.302, std=0.220
---------



2023-12-14 10:51:40,280:CRITICAL:EmoV2_step40: iteration: 210: Loss: 0.249970, lr: 0.009000
2023-12-14 10:51:43,998:CRITICAL:EmoV2_step40: iteration: 220: Loss: 0.260082, lr: 0.009000
2023-12-14 10:51:47,715:CRITICAL:EmoV2_step40: iteration: 230: Loss: 0.261398, lr: 0.009000
2023-12-14 10:51:51,480:CRITICAL:EmoV2_step40: iteration: 240: Loss: 0.250105, lr: 0.009000
2023-12-14 10:51:55,744:CRITICAL:EmoV2_step40: iteration: 250: Loss: 0.270120, lr: 0.009000
2023-12-14 10:52:01,338:CRITICAL:EmoV2_step40: iteration: 260: Loss: 0.262289, lr: 0.009000
2023-12-14 10:52:05,551:CRITICAL:EmoV2_step40: iteration: 270: Loss: 0.264325, lr: 0.009000
2023-12-14 10:52:09,303:CRITICAL:EmoV2_step40: iteration: 280: Loss: 0.260171, lr: 0.009000
2023-12-14 10:52:13,013:CRITICAL:EmoV2_step40: iteration: 290: Loss: 0.249618, lr: 0.009000
2023-12-14 10:52:16,813:CRITICAL:EmoV2_step40: iteration: 300: Loss: 0.258202, lr: 0.008999
2023-12-14 10:54:04,644:CRITICAL:EmoV2_step40: validate. Iteration: 300: Accurac

targets = (38261, 2), predict = (38261, 2)
target = torch.Size([38261]), pred = torch.Size([38261])
Valence:
   accuracy per frames: 2.339%
   error per frames: frames=0.437, std=0.216
   concordance correlation coefficient per frames: nan
target = torch.Size([38261]), pred = torch.Size([38261])
Arousal:
   accuracy per frames: 20.935%
   error per frames: frames=0.318, std=0.216
   concordance correlation coefficient per frames: nan
---------

targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Valence:
   accuracy per clips: 2.970%
   error per clips: mean=0.441, std=0.212
targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Arousal:
   accuracy per clips: 21.782%
   error per clips: mean=0.302, std=0.220
---------



2023-12-14 10:54:07,888:CRITICAL:EmoV2_step40: iteration: 310: Loss: 0.272345, lr: 0.008999
2023-12-14 10:54:11,538:CRITICAL:EmoV2_step40: iteration: 320: Loss: 0.253944, lr: 0.008999
2023-12-14 10:54:15,252:CRITICAL:EmoV2_step40: iteration: 330: Loss: 0.262362, lr: 0.008999
2023-12-14 10:54:18,907:CRITICAL:EmoV2_step40: iteration: 340: Loss: 0.250267, lr: 0.008999
2023-12-14 10:54:22,628:CRITICAL:EmoV2_step40: iteration: 350: Loss: 0.258819, lr: 0.008999
2023-12-14 10:54:26,336:CRITICAL:EmoV2_step40: iteration: 360: Loss: 0.256037, lr: 0.008999
2023-12-14 10:54:29,988:CRITICAL:EmoV2_step40: iteration: 370: Loss: 0.269109, lr: 0.008999
2023-12-14 10:54:33,629:CRITICAL:EmoV2_step40: iteration: 380: Loss: 0.236645, lr: 0.008999
2023-12-14 10:54:37,287:CRITICAL:EmoV2_step40: iteration: 390: Loss: 0.259136, lr: 0.008999
2023-12-14 10:54:40,958:CRITICAL:EmoV2_step40: iteration: 400: Loss: 0.275724, lr: 0.008999
2023-12-14 10:56:28,377:CRITICAL:EmoV2_step40: validate. Iteration: 400: Accurac

targets = (38261, 2), predict = (38261, 2)
target = torch.Size([38261]), pred = torch.Size([38261])
Valence:
   accuracy per frames: 2.339%
   error per frames: frames=0.437, std=0.216
   concordance correlation coefficient per frames: nan
target = torch.Size([38261]), pred = torch.Size([38261])
Arousal:
   accuracy per frames: 20.935%
   error per frames: frames=0.318, std=0.216
   concordance correlation coefficient per frames: nan
---------

targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Valence:
   accuracy per clips: 2.970%
   error per clips: mean=0.441, std=0.212
targetCLIPS = torch.Size([202]), predCLIPS = torch.Size([202])
Arousal:
   accuracy per clips: 21.782%
   error per clips: mean=0.302, std=0.220
---------

